In [15]:
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
from sklearn.preprocessing import StandardScaler
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils.convert import from_networkx, to_networkx
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [16]:
nabil = pd.read_csv("./data/fundamental data/commercial bank/Nabil Bank Limited ( NABIL ).csv")
himal = pd.read_csv("./data/fundamental data/commercial bank/Himalayan Bank Limited ( HBL ).csv")
nabil = nabil.drop(columns=['status'])
himal = himal.drop(columns=['status'])
nabil["per_change"] = nabil["per_change"].fillna(0)
himal["per_change"] = himal["per_change"].fillna(0)
nabil['published_date'] = pd.to_datetime(nabil['published_date'])
himal['published_date'] = pd.to_datetime(himal['published_date'])
nabil.set_index('published_date', inplace=True)
himal.set_index('published_date', inplace=True)
himal = himal[himal.index >= nabil.index[0]]

In [17]:
window_size = 30
step_size = 20
batch_size = 32
vis_col = 'close'
l = len(nabil)

In [18]:
nabil_frames = []
nabil_dates = nabil.index
nabil_values = nabil.values

himal_frames = []
cols = himal.columns

targets = []
for i in range(0, l, step_size):
    end = i + window_size
    if end > l:
        nabil_frames.append(nabil[l-window_size-1: l - 1])
        targets.append(nabil_values[l - 1])

        start_date = nabil_dates[l - window_size - 1]
        end_date = nabil_dates[l - 2]

        himal_frame = himal[himal.index >= start_date]
        himal_frame = himal_frame[himal_frame.index <= end_date]

        if himal_frame is not None:
            himal_frames.append(himal_frame)
        else:
            himal_frames.append(pd.DataFrame(columns=[cols]))
        break

    nabil_frames.append(nabil[i:end])
    targets.append(nabil_values[end])

    start_date = nabil_dates[i]
    end_date = nabil_dates[end-1]

    himal_frame = himal[himal.index >= start_date]
    himal_frame = himal_frame[himal_frame.index <= end_date]

    if himal_frame is not None:
        himal_frames.append(himal_frame)
    else:
        himal_frames.append(pd.DataFrame(columns=[cols]))

In [19]:
nabil_edge_indexes = [from_networkx(nx.visibility_graph(frame[vis_col])).edge_index for frame in nabil_frames]
nabil_frame_dates = [frame.index.strftime('%Y%m%d').astype(int).tolist() for frame in nabil_frames]
himal_edge_indexes = [from_networkx(nx.visibility_graph(frame[vis_col])).edge_index for frame in himal_frames]
himal_frame_dates = [frame.index.strftime('%Y%m%d').astype(int).tolist() for frame in himal_frames]

In [ ]:
nabil_graphs = []
himal_graphs = []
graphs = []
for i in range(len(nabil_frames)):
    nabil_x = torch.tensor(nabil_frames[i].values)
    nabil_dates = torch.tensor(nabil_frame_dates[i])
    himal_x = torch.tensor(himal_frames[i].values)
    himal_dates = torch.tensor(himal_frame_dates[i])

    nabil_graph = Data(x=nabil_x, edge_index=nabil_edge_indexes[i], dates=nabil_dates)
    himal_graph = Data(x=himal_x, edge_index=himal_edge_indexes[i], dates=himal_dates)

    dates1_unique = nabil_graph.dates.unique()
    dates2_unique = himal_graph.dates.unique()

    common_dates = torch.tensor([date for date in dates1_unique if date in dates2_unique])

    new_edge_index = []
    for date in common_dates:
        nodes_in_graph1 = (nabil_graph.dates == date).nonzero(as_tuple=True)[0]
        nodes_in_graph2 = (himal_graph.dates == date).nonzero(as_tuple=True)[0]

        for node1 in nodes_in_graph1:
            for node2 in nodes_in_graph2:
                new_edge_index.append([node1.item(), node2.item() + nabil_graph.x.size(0)])

    new_edge_index = torch.tensor(new_edge_index).t().contiguous()
    x_combined = torch.cat([nabil_graph.x, himal_graph.x], dim=0)

    edge_index_combined = torch.cat([nabil_graph.edge_index, himal_graph.edge_index + nabil_graph.x.size(0), new_edge_index], dim=1)

    graphs.append(Data(x=x_combined, edge_index=edge_index_combined))

In [ ]:
G = to_networkx(graphs[0])

pos = nx.spring_layout(G)

x_coords = [pos[node][0] for node in G.nodes()]
y_coords = [pos[node][1] for node in G.nodes()]

edge_x = []
edge_y = []
for edge in G.edges():
    edge_x.extend([pos[edge[0]][0], pos[edge[1]][0], None])
    edge_y.extend([pos[edge[0]][1], pos[edge[1]][1], None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=x_coords, y=y_coords,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Feature',
            xanchor='left',
            titleside='right'
        ),
    )
)

node_trace.text = [f"Node {node}" for node in G.nodes()]

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Interactive Graph Visualization',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)
                ))

fig.show()